<a href="https://colab.research.google.com/github/mspendyala/pendyala_ai/blob/main/FunctionCalling%20/FunctionCallingUsingPostgresql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install PostgreSQL and psycopg2
!apt-get install -y postgresql postgresql-contrib
!pip install psycopg2-binary
!pip install openai python-dotenv


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common
  postgresql-contrib ssl-cert sysstat
0 upgraded, 14 newly installed, 0 to remove and 45 not upgraded.
Need to get 18.4 MB of archives.
After this operation, 51.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubu

In [27]:
# Start the PostgreSQL service
!sudo service postgresql start


 * Starting PostgreSQL 14 database server
   ...done.


In [28]:
# Locate the pg_hba.conf file
!find /etc/postgresql/ -name pg_hba.conf


/etc/postgresql/14/main/pg_hba.conf


In [37]:
# Modify the pg_hba.conf to use md5 authentication
!sudo sed -i 's/local\s*all\s*postgres\s*peer/local all postgres md5/' /etc/postgresql/14/main/pg_hba.conf


In [38]:
# Restart PostgreSQL service
!sudo service postgresql restart


 * Restarting PostgreSQL 14 database server
   ...done.


In [39]:
# Set a password for the postgres user using psql
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'yourpassword';"


Password for user postgres: ^C


In [41]:
import psycopg2

# Connect to PostgreSQL using the password
conn = psycopg2.connect("dbname=postgres user=postgres password='yourpassword'")
conn.autocommit = True
cur = conn.cursor()

# Create a new database
cur.execute("DROP DATABASE IF EXISTS test_db;")
cur.execute("CREATE DATABASE test_db;")
cur.close()
conn.close()

# Connect to the new database and set up the table
conn = psycopg2.connect("dbname=test_db user=postgres password='yourpassword'")
cur = conn.cursor()

# Create a table and insert sample data
cur.execute("DROP TABLE IF EXISTS test_table;")
cur.execute("CREATE TABLE test_table (id SERIAL PRIMARY KEY, name VARCHAR(50), age INT);")
cur.execute("INSERT INTO test_table (name, age) VALUES ('Alice', 30), ('Bob', 25), ('Charlie', 35);")

# Commit changes and close the connection
conn.commit()
cur.close()
conn.close()


None


In [65]:
import os

# Set the environment variables manually
os.environ['OPENAI_API_KEY'] = 'sk-dUjhajfhjsdhfjdhsfhsjdfhjfdsh'
os.environ['DB_HOST'] = 'localhost'
os.environ['DB_NAME'] = 'test_db'
os.environ['DB_USER'] = 'postgres'
os.environ['DB_PASSWORD'] = 'yourpassword'
os.environ['DB_PORT'] = '5432'



In [66]:
import psycopg2

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host=os.getenv('DB_HOST'),
    database=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    port=os.getenv('DB_PORT')
)

# Function to execute a SQL query
def execute_query(query):
    with conn.cursor() as cur:
        cur.execute(query)
        results = cur.fetchall()
    return results


In [67]:
# Import necessary libraries
from openai import OpenAI
import json
import os
import psycopg2

# Retrieve the API key from the environment variable
openai_api_key = os.getenv('OPENAI_API_KEY')

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=openai_api_key)

# Function to execute a SQL SELECT query
def select_from_database(query):
    with conn.cursor() as cur:
        cur.execute(query)
        results = cur.fetchall()
        # Format the results as a list of dictionaries
        columns = [desc[0] for desc in cur.description]
        results_list = [dict(zip(columns, row)) for row in results]
    return json.dumps(results_list)

def run_database_query_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "Fetch the first 10 rows from the test_table table."}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "select_from_database",
                "description": "Execute a SQL SELECT query on a PostgreSQL database.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The SQL SELECT query to execute.",
                        }
                    },
                    "required": ["query"],
                },
            }
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        available_functions = {
            "select_from_database": select_from_database
        }
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            # Call the function with appropriate arguments
            if function_name == "select_from_database":
                function_response = function_to_call(
                    query=function_args.get("query")
                )

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
    return "No function calls were needed."

# Run the conversation and print the result
database_query_conversation = run_database_query_conversation()

# Print the content of the final response message
if isinstance(database_query_conversation, str):
    print(database_query_conversation)
else:
    print(database_query_conversation.choices[0].message.content)


Here are the first 10 rows from the `test_table`:

1. **ID:** 1, **Name:** Alice, **Age:** 30
2. **ID:** 2, **Name:** Bob, **Age:** 25
3. **ID:** 3, **Name:** Charlie, **Age:** 35

It appears only 3 rows are available in the table. If you need more details or have another request, let me know!
